rdds used years ago before dataframe exist, it more low level

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/25 10:56:28 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.21.134.10 instead (on interface eth0)
23/03/25 10:56:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/25 10:56:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2

In [3]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [4]:
from datetime import datetime
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [5]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 3, 19, 0, 1), PULocationID=244, total_amount=8.8)

In [6]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [7]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [8]:
from collections import namedtuple
def unwrap(row):
    RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [9]:
from pyspark.sql import types
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

### Main stage

In [10]:
#where
# make new variable count
# grouping
# formatting

df_result = rdd\
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [11]:
df_result.write.parquet('data/tmp/green-revenue')

23/03/25 10:56:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/03/25 10:56:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/25 10:56:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/03/25 10:56:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/25 10:56:53 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
23/03/25 10:57:07 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
23/03/25 10:57:07 WARN MemoryManager: Total allocation exceeds 95.

23/03/25 10:57:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/03/25 10:57:08 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


### Map Partitions

In [12]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

duration_rdd = df_green \
    .select(columns) \
    .rdd

In [13]:
import pandas as pd

In [14]:
rows = duration_rdd.take(10)

In [15]:
df = pd.DataFrame(rows, columns=columns)

In [16]:
df

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,1.0,2020-01-03 19:00:01,244,116,1.00
1,2.0,2020-01-29 19:55:01,166,24,0.82
2,1.0,2020-01-02 10:20:42,145,145,0.00
3,NaN,2020-01-23 10:24:00,101,203,4.88
4,2.0,2020-01-12 17:30:50,74,74,0.34
5,2.0,2020-01-27 19:33:09,134,135,1.59
6,2.0,2020-01-24 21:25:12,42,42,0.74
7,2.0,2020-01-04 17:06:40,82,223,4.34
8,NaN,2020-01-07 05:46:00,222,80,5.74
9,NaN,2020-01-08 17:18:00,226,167,6.50


In [17]:
columns

['VendorID',
 'lpep_pickup_datetime',
 'PULocationID',
 'DOLocationID',
 'trip_distance']

In [18]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [19]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [20]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [21]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|               5.0|
|               4.1|
|               0.0|
|              24.4|
|1.7000000000000002|
|              7.95|
|               3.7|
|              21.7|
|28.700000000000003|
|              32.5|
|               4.9|
|             13.25|
|              13.4|
|               0.0|
|             52.75|
|             26.25|
|             31.75|
|              17.5|
|              10.2|
|              14.0|
+------------------+
only showing top 20 rows

